<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Project Title: Youtube Videos Recommendation System Using Bert Embeddings</b></h1>



### Tabel of Contents:

* [Import Libraries](#1)
* [Load Dataset](#2)
* [Bert Model](#4)
* [Predictive for Test](#8)
* [Like this? Upvote and comment! 🌊 End](#6)

**<a id="1"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Import Libraries</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [1]:
!pip install datasets

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import torch
from datasets import load_dataset, Dataset,DatasetDict
from transformers import BertTokenizer, TFBertModel
import re
from sklearn.metrics.pairwise import cosine_similarity
import wandb
wandb.init(mode = "disabled")
import warnings
warnings.filterwarnings('ignore')

**<a id="2"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Load Dataset</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [3]:
# Load the CSV file
df_yt = pd.read_csv('/kaggle/input/top-trending-videos-youtube-2021/US_videos_data.csv')
df_yt = df_yt[['title','channelTitle','likes','dislikes','thumbnail_link','description']]
df_yt.head()

,title,channelTitle,likes,dislikes,thumbnail_link,description
0,Meatpacking: Last Week Tonight with John Olive...,LastWeekTonight,60111,1221,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,The pandemic has thrown into high relief some ...
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",ZHC Crafts,50662,690,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,I can't believe we made art using 3d pens and ...
2,100 Days - [Minecraft Superflat],Luke TheNotable,372753,7961,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,This video is intended for audiences 13+ years...
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,American Idol,14267,129,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,Small but MIGHTY! Casey Bishop completely blow...
4,Game Theory: Did Reddit Just SOLVE FNAF?,The Game Theorists,225780,2872,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,Get Yourself Some BRAND NEW Theory Wear! ► htt...


In [4]:
df_yt.shape

(6000, 6)

In [5]:
df_yt.isna().sum()

title              0
channelTitle       0
likes              0
dislikes           0
thumbnail_link     0
description       15
dtype: int64

In [6]:
df_yt.dropna(inplace=True)

In [7]:
df_yt['clean_title'] = df_yt['title'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x) if isinstance(x, str) else '')
df_yt.head()

,title,channelTitle,likes,dislikes,thumbnail_link,description,clean_title
0,Meatpacking: Last Week Tonight with John Olive...,LastWeekTonight,60111,1221,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,The pandemic has thrown into high relief some ...,Meatpacking Last Week Tonight with John Oliver...
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",ZHC Crafts,50662,690,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,I can't believe we made art using 3d pens and ...,Best 3D Pen Art Wins 5000 Challenge ZHC Crafts
2,100 Days - [Minecraft Superflat],Luke TheNotable,372753,7961,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,This video is intended for audiences 13+ years...,100 Days Minecraft Superflat
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,American Idol,14267,129,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,Small but MIGHTY! Casey Bishop completely blow...,Amazing Luke Bryan Calls 15YearOld Casey Bisho...
4,Game Theory: Did Reddit Just SOLVE FNAF?,The Game Theorists,225780,2872,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,Get Yourself Some BRAND NEW Theory Wear! ► htt...,Game Theory Did Reddit Just SOLVE FNAF


<a id="4"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Bert Model</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [8]:
# Load the Hugging Face BERT tokenizer and model for TensorFlow
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [9]:
input_max_len = max(len(tokenizer.encode(text)) for text in df_yt["title"])
print(f"Calculated Input max_length: {input_max_len}")

Calculated Input max_length: 100


In [10]:
def get_bert_embeddings(text, tokenizer, model):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
    # Use the Hugging Face TFBertModel to get the embeddings
    outputs = model(inputs)
    # Return the pooled output (embedding for the [CLS] token)
    return outputs.pooler_output.numpy()
# Apply the function to each cleaned title in the DataFrame
df_yt['embeddings'] = df_yt['clean_title'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))

In [11]:
df_yt.head()

,title,channelTitle,likes,dislikes,thumbnail_link,description,clean_title,embeddings
0,Meatpacking: Last Week Tonight with John Olive...,LastWeekTonight,60111,1221,https://i.ytimg.com/vi/IhO1FcjDMV4/default.jpg,The pandemic has thrown into high relief some ...,Meatpacking Last Week Tonight with John Oliver...,"[[-0.8381992, -0.4944186, -0.60153407, 0.63540..."
1,"Best 3D Pen Art Wins $5,000 Challenge! | ZHC C...",ZHC Crafts,50662,690,https://i.ytimg.com/vi/p4Wy84AOzj0/default.jpg,I can't believe we made art using 3d pens and ...,Best 3D Pen Art Wins 5000 Challenge ZHC Crafts,"[[-0.8337095, -0.4984601, -0.80845195, 0.68989..."
2,100 Days - [Minecraft Superflat],Luke TheNotable,372753,7961,https://i.ytimg.com/vi/4eKXwKDdXYA/default.jpg,This video is intended for audiences 13+ years...,100 Days Minecraft Superflat,"[[-0.82278955, -0.38171282, -0.63249546, 0.718..."
3,Amazing! Luke Bryan Calls 15-Year-Old Casey Bi...,American Idol,14267,129,https://i.ytimg.com/vi/XHR5mt2gBjo/default.jpg,Small but MIGHTY! Casey Bishop completely blow...,Amazing Luke Bryan Calls 15YearOld Casey Bisho...,"[[-0.8599135, -0.31500268, 0.1849855, 0.530873..."
4,Game Theory: Did Reddit Just SOLVE FNAF?,The Game Theorists,225780,2872,https://i.ytimg.com/vi/C-icyHEb7W4/default.jpg,Get Yourself Some BRAND NEW Theory Wear! ► htt...,Game Theory Did Reddit Just SOLVE FNAF,"[[-0.8647456, -0.41429687, -0.582271, 0.714787..."


In [12]:
# save fianal df
df_yt.to_csv('final_df.csv', index=False)

<a id="8"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 20px 10px 10px #042b4c">Predictive for Test</p></b></h1>

In [13]:

def compute_cosine_similarity(embedding, embeddings):
    """
    Compute cosine similarity between a single embedding and all other embeddings.

    Args:
    - embedding (np.ndarray): The embedding vector for the input title.
    - embeddings (list of np.ndarray): List of all embedding vectors in the dataset.

    Returns:
    - similarities (np.ndarray): Array of cosine similarities.
    """
    similarities = cosine_similarity(embedding.reshape(1, -1), np.vstack(embeddings)).flatten()
    return similarities

def recommend_videos(title, df, tokenizer, model, top_n=5):
    """
    Recommend top N similar videos based on the embeddings of the cleaned titles.

    Args:
    - title (str): The title of the video for which we want to find similar videos.
    - df (pd.DataFrame): DataFrame containing video data and embeddings.
    - tokenizer (BertTokenizer): Hugging Face tokenizer.
    - model (TFBertModel): Hugging Face BERT model.
    - top_n (int): Number of similar videos to recommend.

    Returns:
    - recommendations (pd.DataFrame): DataFrame of recommended videos.
    """
    # Preprocess and get the embedding for the input title
    cleaned_title = re.sub('[^A-Za-z0-9]+', ' ', title.lower())
    embedding = get_bert_embeddings(cleaned_title, tokenizer, model)

    # Compute similarities between the input embedding and all other embeddings
    similarities = compute_cosine_similarity(embedding, df['embeddings'].tolist())

    # Add the similarity scores to the DataFrame
    df['similarity'] = similarities

    # Sort the DataFrame based on similarity scores in descending order
    df_sorted = df.sort_values(by='similarity', ascending=False)

    # Return the top N recommendations (excluding the input title itself if it's present)
    recommendations = df_sorted[df_sorted['title'] != title].head(top_n)

    return recommendations[['title', 'channelTitle','likes','dislikes','thumbnail_link', 'similarity']]

In [14]:
def display_recommendations(recommendations):
    # Generate HTML for displaying thumbnails and metadata
    html = '<div style="display: flex; flex-wrap: wrap; justify-content: space-around;">'

    # Iterate over recommendations in pairs (2 thumbnails per row)
    for i in range(0, len(recommendations), 2):
        row_html = '<div style="display: flex; justify-content: space-around; width: 100%;">'

        # Iterate through each item in the pair (2 thumbnails per row)
        for j in range(2):
            if i + j < len(recommendations):
                # Fetch data for each recommendation
                recommendation = recommendations.iloc[i + j]
                title = recommendation['title']
                thumbnail = recommendation['thumbnail_link']
                likes = recommendation['likes']
                dislikes = recommendation['dislikes']
                similarity = recommendation['similarity']
                channelTitle = recommendation['channelTitle']

                # Generate HTML for each video thumbnail and details
                row_html += f'''
                <div style="width: 20%; margin: 2px; text-align: center; border: 1px solid #ddd; padding: 2px; border-radius: 10px;">
                    <img src="{thumbnail}" alt="{title}" style="width: 100%; border-radius: 5px;">
                    <h4>{title}</h4>
                    <p>Channel: {channelTitle}</p>
                    <p>Likes: {likes} | Dislikes: {dislikes}</p>
                    <p>Similarity: {similarity:.2f}</p>
                </div>
                '''
        row_html += '</div>'
        html += row_html

    html += '</div>'
    display(HTML(html))

In [15]:
top_recommendations = recommend_videos("funny videos", df_yt, tokenizer, model, top_n=10)
top_recommendations

,title,channelTitle,likes,dislikes,thumbnail_link,similarity
237,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,184540,1492,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
641,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,337301,4240,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
421,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,293902,3139,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
869,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,390271,8972,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
1564,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,453696,13685,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
1317,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,442070,13526,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
1085,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,선미 SUNMI,425702,13374,https://i.ytimg.com/vi/Aui0ZKIaXVc/default.jpg,0.992624
184,Something About Smash Bros THE SUBSPACE EMISSA...,TerminalMontage,407664,3003,https://i.ytimg.com/vi/5juBmmXoW7I/default.jpg,0.991469
4121,Happy Mario Day!,ScottFalco,52466,383,https://i.ytimg.com/vi/476Mm0T52NM/default.jpg,0.991445
3650,Happy Mario Day!,ScottFalco,49482,355,https://i.ytimg.com/vi/476Mm0T52NM/default.jpg,0.991445


In [16]:
display_recommendations(top_recommendations)

In [17]:
top_recommendations = recommend_videos("Romantic videos", df_yt, tokenizer, model, top_n=5)
top_recommendations

,title,channelTitle,likes,dislikes,thumbnail_link,similarity
1385,MASHALLAH (Official Music Video) - fousey x Ad...,FOUSEY,17010,1670,https://i.ytimg.com/vi/ArncdwDE040/default.jpg,0.993152
932,MASHALLAH (Official Music Video) - fousey x Ad...,FOUSEY,16455,1517,https://i.ytimg.com/vi/ArncdwDE040/default.jpg,0.993152
36,MASHALLAH (Official Music Video) - fousey x Ad...,FOUSEY,11952,545,https://i.ytimg.com/vi/ArncdwDE040/default.jpg,0.993152
694,MASHALLAH (Official Music Video) - fousey x Ad...,FOUSEY,15835,1439,https://i.ytimg.com/vi/ArncdwDE040/default.jpg,0.993152
265,MASHALLAH (Official Music Video) - fousey x Ad...,FOUSEY,14516,1242,https://i.ytimg.com/vi/ArncdwDE040/default.jpg,0.993152


In [18]:
display_recommendations(top_recommendations)

<center><span style="font-family:Palatino; font-size:22px;"><i>Like this? <span style="color:#DC143C;">Upvote and Comment!</span> </i>🌊 End</span> </center>